<a href="https://colab.research.google.com/github/helvete74/IFAGE_dev_AI/blob/main/projet_head_detection_V1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Head Detection V1.5

https://universe.roboflow.com/head-znpny/head-detection-nej1a/

charge le dataset au YoloV8
model pre-entrainé : yolo_v8_XXX

différents essais pour améliorer la detection/ supprimer les doubles detections


# Hyperparameters

In [ ]:
YOLOV8BACKBONE = "yolo_v8_s_backbone"
# YOLOV8BACKBONE = "yolo_v8_s_backbone_coco"
# YOLOV8BACKBONE = "yolo_v8_m_backbone_coco"
# YOLOV8BACKBONE = "yolo_v8_l_backbone"

BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 70
GLOBAL_CLIPNORM = 10.0

BOUNDING_BOX_FORMAT = "xyxy"

IOU_THRESHOLD = 0.5
CONFIDENCE_THRESHOLD = 0.75

#Installation et import

In [ ]:
!pip install -U wandb
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

wandb.login(key="ea33b1bdf2e44dad5cdbc06372974ee5207fd710")
# API key : ea33b1bdf2e44dad5cdbc06372974ee5207fd710

In [ ]:
!pip install roboflow

!pip list| grep roboflow

In [ ]:
!pip install --upgrade git+https://github.com/keras-team/keras-cv -q

In [ ]:
# check install keras
!pip list| grep keras

In [ ]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

from keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

# Creation du Dataset

## Téléchargement depuis roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="4RTem5JDv2kRxw8jk1El")
project = rf.workspace("head-znpny").project("head-detection-nej1a")
version = project.version(1)
roboflow_dataset = version.download("voc")



model = version.model
str(model)

In [ ]:
roboflow_dataset.location
roboflow_dataset.model_format
roboflow_dataset.name
roboflow_dataset.version


print(roboflow_dataset.location)
data_path = roboflow_dataset.location

en format voc, structure des fichiers :\
head-detection-1
*   test
  *   file.jpg
  *   file.xml
*   train
  *   file.jpg
  *   file.xml
*   valid
  *   file.jpg
  *   file.xml


## Manipulation de fichier

In [ ]:
# suppression de sample_data
!rm -rfv sample_data

## Manipulation des datas

A dictionary is created to map each class name to a unique numerical identifier. This mapping is used to encode and decode the class labels during training and inference in object detection tasks.\

The

In [ ]:
# class_ids from the Udacity Self Driving Car Dataset
class_ids = [
    "0",
    "null"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))
class_mapping

In [ ]:
# Path to images and annotations
# path_images = "/kaggle/input/dataset/data/images/"
# path_annot = "/kaggle/input/dataset/data/annotations/"
# data_path = /content/head-detection-1


def parse_directory(path_data):
  # Get all XML file paths in path_annot and sort them
  xml_files = sorted(
      [
          os.path.join(path_data, file_name)
          for file_name in os.listdir(path_data)
          if file_name.endswith(".xml")
      ]
  )

  # Get all JPEG image file paths in path_images and sort them
  jpg_files = sorted(
      [
          os.path.join(path_data, file_name)
          for file_name in os.listdir(path_data)
          if file_name.endswith(".jpg")
      ]
  )
  return xml_files, jpg_files


path_data_train = os.path.join(data_path, "train")
path_data_valid = os.path.join(data_path, "valid")

xml_files_train, jpg_files_train = parse_directory(path_data_train)
xml_files_valid, jpg_files_valid = parse_directory(path_data_valid)

Nombre d'image du dataset :\
roboflow annonce 1251

In [ ]:
len(xml_files_train)

In [ ]:
len(jpg_files_train)

check the annotation files for one image


In [ ]:
# l'image
# filepath1 = "/content/head-detection-1/train/2015_05_08_07_54_19FrontColor-aviImagen6_jpg.rf.70e018b14f081a5fbc5313cd11f24e3b.jpg"
# image1 = keras.utils.load_img(filepath1)

# l'annotation
# xml_file1 = "/content/head-detection-1/train/2015_05_08_07_54_19FrontColor-aviImagen6_jpg.rf.70e018b14f081a5fbc5313cd11f24e3b.xml"
# tree1 = ET.parse(xml_file1)
# root1 = tree1.getroot()
# xml_str1 = ET.tostring(root1, encoding='unicode')
# print("filename : ", root1.find("filename").text)
# print("xml_str : ", xml_str1)


cree:
```
bounding_boxes = {
    # num_boxes may be a Ragged dimension
    'boxes': Tensor(shape=[batch, num_boxes, 4]),
    'classes': Tensor(shape=[batch, num_boxes])
}
```



In [ ]:
def parse_annotation(path_data, xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_data, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids

def create_bounding_boxes(path_data, xml_files):
  image_paths = []
  bbox = []
  classes = []
  for xml_file in tqdm(xml_files):
      image_path, boxes, class_ids = parse_annotation(path_data, xml_file)
      image_paths.append(image_path)
      bbox.append(boxes)
      classes.append(class_ids)

  # create irregular tensor with tf.ragged
  bbox = tf.ragged.constant(bbox)
  classes = tf.ragged.constant(classes)
  image_paths = tf.ragged.constant(image_paths)

  return image_paths, bbox, classes


image_paths_train, bbox_train, classes_train = create_bounding_boxes(path_data_train, xml_files_train)
image_paths_valid, bbox_valid, classes_valid = create_bounding_boxes(path_data_valid, xml_files_valid)


In [ ]:
# classes

In [ ]:
# create the dataset
data_train = tf.data.Dataset.from_tensor_slices((image_paths_train, classes_train, bbox_train))
data_valid = tf.data.Dataset.from_tensor_slices((image_paths_valid, classes_valid, bbox_valid))


## Splitting data in training and validation data

In [ ]:
# plus de splitting, on se base sur le splitting du dataset

Train des dataset train_data et val_data

In [ ]:
print (f"Nombre d'élément du dataset train_data : {len(data_train)}")
print (f"Nombre d'élément de dataset val_data : {len(data_valid)}")

Data Augmentation

In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}



In [ ]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="xyxy"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xyxy"
        ),
    ]
)


In [ ]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)

Creating Training Dataset

In [ ]:
train_ds = data_train.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)

Creating Validation Dataset

In [ ]:
val_ds = data_valid.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

## Visualization

In [ ]:
BOUNDING_BOX_FORMAT = "xyxy"
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )

visualize_dataset(
    train_ds, bounding_box_format=BOUNDING_BOX_FORMAT, value_range=(0, 255), rows=2, cols=2
)

visualize_dataset(
    val_ds, bounding_box_format=BOUNDING_BOX_FORMAT, value_range=(0, 255), rows=2, cols=2
)

We need to extract the inputs from the preprocessing dictionary and get them ready to be fed into the model.

In [ ]:
def dict_to_tuple(inputs):
    #return inputs["images"], inputs["bounding_boxes"]  # init
    return inputs["images"], bounding_box.to_dense(
        inputs["bounding_boxes"], max_boxes=32
    )


train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
train_ds

# Entrainement par transfert

List des models YoloV8 :  https://keras.io/api/keras_cv/models/backbones/yolo_v8/

##Téléchargement d'un model YoloV8 pre-entrainé

In [ ]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    YOLOV8BACKBONE  # see YOLOV8BACKBONE
)

integration de nos paramètres :

In [ ]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format=BOUNDING_BOX_FORMAT,
    backbone=backbone,
    fpn_depth=1,
)

Compile the Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

yolo.compile(
    optimizer=optimizer,
    classification_loss="binary_crossentropy",
    box_loss="ciou",
)


## Train the Model

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Head détection V1.5 YOLO_S EPOCH",

    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "global_clipnorm" : GLOBAL_CLIPNORM,
    "YOLOV8Backbone": YOLOV8BACKBONE,
    "dataset": "https://universe.roboflow.com/head-znpny/head-detection-nej1a/dataset/1",
    "epochs": EPOCH,
    "batch_size": BATCH_SIZE,
    "num_classes": len(class_mapping),
    "bounding_box_format": BOUNDING_BOX_FORMAT,
    "augmentation": "Yes",
    "early_stopping": "No",
    }
)

In [ ]:
coco_metrics_callback = keras_cv.callbacks.PyCOCOCallback(
    val_ds.take(20), bounding_box_format=BOUNDING_BOX_FORMAT
)

# https://keras.io/api/callbacks/early_stopping/
early_stopping = EarlyStopping(
    monitor="box_loss",
    min_delta=1e-4,
    patience=3,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=20,
)

'''
# https://keras.io/api/callbacks/model_checkpoint/
model_checkpoint = ModelCheckpoint(
    filepath=config["checkpoint_path"],
    save_best_only=True,
    verbose=1
)
'''


In [ ]:
print("Fit model on training data")
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCH,
    callbacks=[
      WandbMetricsLogger(),
      WandbModelCheckpoint("models"),

    ]
)

# coco_metrics_callback
# early_stopping

In [ ]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

In [ ]:
# The returned history object holds a record of the loss values and metric values during training:
print(yolo.history.history)

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["loss"])
    plt.plot(hist.history["box_loss"])
    plt.title("model box_loss")
    plt.ylabel("box_loss")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

plot_hist(yolo.history)

## Sauvegarde du model après entrainement

In [ ]:
# yolo.save("myYolo_HeadDetect_model")

# Utilisation de notre model entrainé

##Load de notre model

In [ ]:
# pour être plus rapide à l'execution, je ne sauve pas le modele, donc je ne le recharge pas
'''
# need to specify the bounding_box_format
BOUNDING_BOX_FORMAT = "xyxy"
custom_objects = {'CIoULoss': keras_cv.losses.CIoULoss}

my_model = keras.models.load_model("myYolo_HeadDetect_model",
                                   custom_objects=custom_objects,
                                   compile=False)

# Compile the model after loading
my_model.compile(optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou")
'''

## Utilisation du model avec une image

On prend une image du dataset
2015_05_08_08_00_40FrontColor-aviImagen3_jpg.rf.13e2ef1b695dc8d63e355a7a205d1ab9.jpg

In [ ]:
import numpy as np

filepath = "/content/head-detection-1/test/2015_05_10_09_43_50FrontColor-aviImagen1_jpg.rf.c6bdcbce984aced0f03031e996121441.jpg"
image = keras.utils.load_img(filepath)
image = np.array(image)

visualization.plot_image_gallery(
    np.array([image]),
    value_range=(0, 255),
    rows=1,
    cols=1,
    scale=5,
)

## Perform detections with my model

In [ ]:
# redimensionnement de l'image avec ratio identique
inference_resizing = keras_cv.layers.Resizing(
    640, 640, pad_to_aspect_ratio=True, bounding_box_format=BOUNDING_BOX_FORMAT
)

image_batch = inference_resizing([image])
len(image_batch)

In [ ]:
# avec notre model en direct
my_model = yolo

y_pred1 = my_model.predict(image_batch)
y_pred1

In [ ]:
# visualizase :
visualization.plot_bounding_box_gallery(
    image_batch,
    value_range=(0, 255),
    rows=1,
    cols=1,
    y_pred=y_pred1,
    scale=5,
    font_scale=0.7,
    bounding_box_format=BOUNDING_BOX_FORMAT,
    class_mapping=class_mapping,
)

On trouve une tête, marche pas a tout les coups


## Validation du model pré-entrainé avec le dataset Test : test_on_batch

### load du dataset de Test

In [ ]:
# load du dataset de Test

# parse file from Test folder
path_annot = "head-detection-1/test"
path_images  = "head-detection-1/test"

# Get all XML file paths in path_annot and sort them
xml_files_T = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files_T = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)

image_paths_T = []
bbox_T = []
classes_T = []
for xml_file in tqdm(xml_files_T):
    image_path_T, boxes_T, class_ids_T = parse_annotation(path_annot, xml_file)
    image_paths_T.append(image_path_T)
    bbox_T.append(boxes_T)
    classes_T.append(class_ids_T)



# create irregular tensor with tf.ragged
classes_T = tf.ragged.constant(classes_T)
bbox_T = tf.ragged.constant(bbox_T)
image_paths_T = tf.ragged.constant(image_paths_T)

# create the dataset
data_T = tf.data.Dataset.from_tensor_slices((image_paths_T, classes_T, bbox_T))

Test_ds = data_T.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
Test_ds = Test_ds.shuffle(BATCH_SIZE * 4)
Test_ds = Test_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
Test_ds = Test_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)



# visualize Test_ds
visualize_dataset(
    Test_ds, bounding_box_format=BOUNDING_BOX_FORMAT, value_range=(0, 255), rows=2, cols=2
)



### Test_on_batch

In [ ]:
''' desactivation pour le moment

# Load and preprocess the images
image_batch = []
input_height = 640
input_width = 640
for image_path in image_paths_T:
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize image if necessary to match your model's input size
    image = tf.image.resize(image, (input_height, input_width))
    image_batch.append(image)

image_batch = tf.stack(image_batch) # Stack images into a batch tensor

# Prepare labels and bounding boxes in the expected format (dictionary)
y_true = {
    "classes": classes_T.to_tensor(),  # Convert RaggedTensor to a regular tensor
    "boxes": bbox_T.to_tensor()       # Convert RaggedTensor to a regular tensor
}

# Now test on the preprocessed image batch, passing the labels in the correct format
my_model.test_on_batch(image_batch, y=y_true, sample_weight=None, return_dict=True)

'''

A scalar loss value (when no metrics and return_dict=False), a list of loss and metric values (if there are metrics and return_dict=False), or a dict of metric and loss values (if return_dict=True).

A comparer avec les stats de notre entrainement

##Visualize Predictions

Exemple dans https://keras.io/examples/vision/yolov8/

In [ ]:
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )

'''
yolo.prediction_decoder = keras_cv.layers.NonMaxSuppression(
    from_logits=True,
    bounding_box_format=BOUNDING_BOX_FORMAT,
    iou_threshold=IOU_THRESHOLD,
    confidence_threshold=0.55
)
'''
#    from_logits=True,
#    iou_threshold=IOU_THRESHOLD,


visualize_detections(yolo, dataset=val_ds, bounding_box_format="xyxy")


## Intersection over Union (IoU) for object detection

https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/\

Intersection over Union (IoU) is used to evaluate the performance of object detection by comparing the ground truth bounding box to the preddicted bounding box

In [ ]:
# DEBUG
Test_ds # dataset avec image et boxes resize

'''
<_ParallelMapDataset
element_spec={
    'images': TensorSpec(shape=(4, 640, 640, 3), dtype=tf.float32, name=None),
    'bounding_boxes': {
        'classes': RaggedTensorSpec(TensorShape([4, None]), tf.float32, 1, tf.int64),
        'boxes': RaggedTensorSpec(TensorShape([4, None, None]), tf.float32, 1, tf.int64)
    }
}>
'''


In [ ]:
# method pour la compile pour reduire le IoU
'''
tf.keras.metrics.MeanIoU(
    num_classes,
    name=None,
    dtype=None,
    ignore_class=None,
    sparse_y_true=True,
    sparse_y_pred=True,
    axis=-1
)
'''

In [ ]:
''' desactivation pour le moment

def visualize_detections_and_compute_iou(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    print(f"Nombre d'y_true: {len(y_true)}")
    y_pred = model.predict(images)
    #y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )

    # No conversion needed, they are already dense tensors
    y_true_boxes_dense = y_true['boxes']
    y_pred_boxes_dense = y_pred['boxes']


    # pour calculer le IoU
    # boxes1 : les boxes du vals_ds
    # boxes2 : les boxes calculées du val_ds
    iou = keras_cv.bounding_box.compute_iou(
        boxes1 = y_true_boxes_dense,
        boxes2 = y_pred_boxes_dense,
        bounding_box_format=bounding_box_format,
        use_masking=False,
        mask_val=-1,
        images=None,
        image_shape=None,
    )
    return iou


iou = visualize_detections_and_compute_iou(yolo, dataset=val_ds, bounding_box_format="xyxy")
print(iou)

'''